## Exploratory Data Analysis

Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA.


We shouldn't limit our EDA to these 10 questions. Let's be creative :).

In [1]:
import pandas as pd
import psycopg2
import requests
import plotly.express as px
import json
import datetime
from IPython.display import JSON

In [2]:
conn = psycopg2.connect(database="mid_term_project", user="lhl_student", password="lhl_student", host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")
print("Database opened successfully")

Database opened successfully


In [144]:
cur = conn.cursor()

#### **Task 1**: Test the hypothesis that the delay is from Normal distribution. and that **mean** of the delay is 0. Be careful about the outliers.

#### **Task 2**: Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**: What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

#### **Task 6**: How many states cover 50% of US air traffic? 

In [3]:
dest_city = pd.read_sql_query('''SELECT COUNT(dest_city_name), dest_city_name FROM flights GROUP BY dest_city_name;''', conn)

In [4]:
dest_city['State'] = dest_city['dest_city_name'].apply(lambda x: x[-2:])

In [5]:
dest_city = dest_city.rename(columns={"count": "flight_count"})


In [6]:
dest = dest_city.groupby('State').sum().sort_values(by ='flight_count',ascending=False).reset_index()

In [138]:
half = 0
top_50 = 0
for _ in dest.flight_count:
    if half <= int(dest.flight_count.sum())/2:
        half += int(dest.flight_count[top_50])
        top_50 += 1

In [7]:
dest

,State,flight_count
0,CA,1738330
1,TX,1572051
2,FL,1151320
3,IL,1008051
4,GA,841095
5,NY,827847
6,NC,736070
7,CO,622441
8,PA,475222
9,DC,472915


In [139]:
top_50

8

#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

In [146]:
which_hour = pd.read_sql_query('''SELECT dep_time, air_time, CASE 
WHEN air_time >=100 THEN 'LONG' 
WHEN air_time >=50 THEN 'MEDIUM' 
WHEN air_time >0 THEN 'SHORT'
ELSE null END AS dur_type 
FROM flights limit 100;''', conn)

In [147]:
which_hour

,dep_time,air_time,dur_type
0,2153.0,32.0,SHORT
1,1418.0,37.0,SHORT
2,1541.0,35.0,SHORT
3,2154.0,42.0,SHORT
4,1949.0,61.0,MEDIUM
...,...,...,...
95,615.0,93.0,MEDIUM
96,1650.0,162.0,LONG
97,1729.0,308.0,LONG
98,1254.0,302.0,LONG


#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?